# Table of Contents

0. <a href="#sec0">Dependencies</a>
1. <a href="#sec1">Reproducing the Paper</a>
2. <a href="#sec2">Training the Model on Custom Datasets</a>
3. <a href="#sec3">Inference using Trained Model</a>

<a id="sec0"></a>
## 0. Dependencies

### Python Packages
The first step after cloning this repository is download and install the necessary python libraries/packages. Install the required packages by running the following cell.

In [ ]:
%pip install -r requirements.txt

<a id="sec1"></a>
## 1. Reproducing the Paper

### 1.1 Download Data
The next step is to download the data used to train/evaluate the models. Running the following command will download all 3 datasets, and convert their encodings so that they can be used by PeptideBERT.

In [ ]:
!python ./data/download_data.py

### 1.2 Train-Val-Test Split
Now, we want to combine the positive and negative samples (downloaded by the above cell), shuffle them and split them into 3 non-overlapping sets - train, validation, and test.

To do so, run the following cell, this will create sub-directories (inside the `data` directory) for each dataset and place the subsets (train, validation, test) inside it.

Additionally, if you want to augment any dataset, you can do so by editing `./data/split_augment.py` file. You can call the `augment_data` function from the `main` function with the dataset that you want to augment. For example, if you want to augment the `solubility` dataset, you can add `augment_data('sol')` to the `main` function.

Further, to change/experiment with the augmentation techniques applied, you can edit the `augment_data` function. Comment/uncomment the call to any of the augmentation functions (such as `random_replace`, `random_delete`, etc.) as desired, change the factor for augmentation as desired. Do keep in mind that for each augmentation applied, you have to call the `combine` function. For example, if you want to apply the `random_swap` augmentation with a `factor` of 0.2, you can add `new_inputs, new_labels = random_swap(inputs, labels, 0.2)` followed by `inputs, labels = combine(inputs, labels, new_inputs, new_labels)` to merge the augmented dataset into the original dataset.

In [ ]:
!python ./data/split_augment.py

### 1.3 Model Config
Edit the `config.yaml` file and set the `task` parameter to one of `hemo` (for hemolysis dataset), `sol` (for solubility dataset), or `nf` (for non-fouling dataset) as desired.

Additionally, If you want to tweak the model before training, you can do so by editing `./model/network.py` and `config.yaml` files. `./model/network.py` contains the actual architecture of the model as well as the optimizer and scheduler used to train the model. `config.yaml` contains all the hyperparameters used for training, as well as which dataset to train on.

### 1.4 Training
Now we are ready to train our model. Run the following cell to start the training procedure. This will save a checkpoint of the best model (on validation set) inside the `checkpoints` directory

In [ ]:
!python train.py

<a id="sec2"></a>
## 2. Training the Model on Custom Datasets

Follow the cells below to train the model on custom datasets.

### 2.1 Data Preparation

create a `csv` file with the following format:
```csv
sequence,label
AAAAAAA,1
LLLLLLL,0
CCCCCCC,0
DDDDDDD,1
```
where `sequence` is the peptide sequence and `label` is the binary label (0 or 1). Save this file as `custom_data.csv` inside the `data` directory. Now, run the following cell (edit `task_name` as desired) to convert the `csv` file to the format required by PeptideBERT.

In [ ]:
import numpy as np

task_name = 'REPLACE_WITH_TASK_NAME'

# read data
seqs, labels = [], []
with open('./data/custom_data.csv', 'r') as f:
    for line in f.readlines()[1:]:
        seq, label = line.strip().split(',')
        seqs.append(seq)
        labels.append(int(label))

MAX_LEN = max(map(len, seqs))

# convert to tokens
mapping = dict(zip(
    ['[PAD]','[UNK]','[CLS]','[SEP]','[MASK]','L',
    'A','G','V','E','S','I','K','R','D','T','P','N',
    'Q','F','Y','M','H','C','W'],
    range(30)
))

pos_data, neg_data = [], []
for i in range(len(seqs)):
    seq = [mapping[c] for c in seqs[i]] 
    seq.extend([0] * (MAX_LEN - len(seq)))  # padding to max length
    if labels[i] == 1:
        pos_data.append(seq)
    else:
        neg_data.append(seq)

pos_data = np.array(pos_data)
neg_data = np.array(neg_data)

np.savez(
    f'./data/{task_name}-positive.npz',
    arr_0=pos_data
)
np.savez(
    f'./data/{task_name}-negative.npz',
    arr_0=neg_data
)

### 2.2 Train-Val-Test Split
Now, we want to combine the positive and negative samples, shuffle them and split them into 3 non-overlapping sets - train, validation, and test.

To do so, edit the `main` function inside `./data/split_augment.py` file (comment existing calls to `split_data` and add the line `split_data('REPLACE_WITH_TASK_NAME')`) and run the following cell, this will create sub-directories (inside the `data` directory) for the custom dataset and place the subsets (train, validation, test) inside it.

Additionally, if you want to augment the dataset, you can do so by editing `./data/split_augment.py` file. You can call the `augment_data` function from the `main` function like so: `augment_data('REPLACE_WITH_TASK_NAME')`.

Further, to change/experiment with the augmentation techniques applied, you can edit the `augment_data` function. Comment/uncomment the call to any of the augmentation functions (such as `random_replace`, `random_delete`, etc.) as desired, change the factor for augmentation as desired. Do keep in mind that for each augmentation applied, you have to call the `combine` function. For example, if you want to apply the `random_swap` augmentation with a `factor` of 0.2, you can add `new_inputs, new_labels = random_swap(inputs, labels, 0.2)` followed by `inputs, labels = combine(inputs, labels, new_inputs, new_labels)` to merge the augmented dataset into the original dataset.

In [ ]:
!python ./data/split_augment.py

### 2.3 Model Config
Edit the `config.yaml` file and set the `task` parameter to `REPLACE_WITH_TASK_NAME`.

Additionally, If you want to tweak the model before training, you can do so by editing `./model/network.py` and `config.yaml` files. `./model/network.py` contains the actual architecture of the model as well as the optimizer and scheduler used to train the model. `config.yaml` contains all the hyperparameters used for training, as well as which dataset to train on.

### 2.4 Training
Now we are ready to train our model. Run the following cell to start the training procedure. This will save a checkpoint of the best model (on validation set) inside the `checkpoints` directory

In [ ]:
!python train.py

<a id="sec3"></a>
## 3. Inference using Trained Model

### 3.1 Load Trained Model
Load the trained model by running the following cell. Edit the `run_name` parameter to the name of the directory containing the trained model (inside the `checkpoints` directory).

In [ ]:
import torch
import yaml
from model.network import create_model

run_name = 'REPLACE_WITH_RUN_NAME'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


config = yaml.load(open('./config.yaml', 'r'), Loader=yaml.FullLoader)
config['device'] = device

model = create_model(config)
model.load_state_dict(torch.load(f'./checkpoints/{run_name}/model.pt')['model_state_dict'], strict=False)

### 3.2 Input Data
Create a text file containing peptide sequences in the following format:
```txt
AAAAAAA
LLLLLLL
CCCCCCC
DDDDDDD
```
where each line represents a peptide sequence. Save this file as `input.txt` inside the `data` directory and run the following cell. The corresponding predictions will be saved in `output.txt` file inside the `data` directory.

In [ ]:
seqs = []
with open('./data/input.txt', 'r') as f:
    for line in f.readlines():
        seq = line.strip()
        seqs.append(seq)

MAX_LEN = max(map(len, seqs))

# convert to tokens
mapping = dict(zip(
    ['[PAD]','[UNK]','[CLS]','[SEP]','[MASK]','L',
    'A','G','V','E','S','I','K','R','D','T','P','N',
    'Q','F','Y','M','H','C','W'],
    range(30)
))

for i in range(len(seqs)):
    seqs[i] = [mapping[c] for c in seqs[i]] 
    seqs[i].extend([0] * (MAX_LEN - len(seqs[i])))  # padding to max length

preds = []
with torch.inference_mode():
    for i in range(len(seqs)):
        input_ids = torch.tensor([seqs[i]]).to(device)
        attention_mask = (input_ids != 0).float()
        output = int(model(input_ids, attention_mask)[0] > 0.5)
        print(output)
        preds.append(output)

with open('./data/output.txt', 'w') as f:
    for pred in preds:
        f.write(str(pred) + '\n')